In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [6]:
# Carga de archivo
ruta_archivo = "proyecto_training_data.npy"
datos = np.load(ruta_archivo)
np.set_printoptions(suppress=True)

# Ver la estructura del dataset
print("Tipo de datos:", type(datos))
print("Forma del dataset:", datos.shape)
print("Primeras filas del dataset:\n", datos[:10])

Tipo de datos: <class 'numpy.ndarray'>
Forma del dataset: (1460, 6)
Primeras filas del dataset:
 [[208500.      7.    856.      8.   2003.     65.]
 [181500.      6.   1262.      6.   1976.     80.]
 [223500.      7.    920.      6.   2001.     68.]
 [140000.      7.    961.      7.   1915.     60.]
 [250000.      8.   1145.      9.   2000.     84.]
 [143000.      5.    796.      5.   1993.     85.]
 [307000.      8.   1694.      7.   2004.     75.]
 [200000.      7.   1107.      7.   1973.     nan]
 [129900.      7.   1022.      8.   1931.     51.]
 [118000.      5.   1077.      5.   1939.     50.]]


In [ ]:
# Dividir en entrenamiento (80%) y validación (20%)
train_data, val_data = train_test_split(datos, test_size=0.2, random_state=2025)

# Verificar tamaños
print("Tamaño del conjunto de entrenamiento:", train_data.shape)
print("Tamaño del conjunto de validación:", val_data.shape)

Tamaño del conjunto de entrenamiento: (1168, 6)
Tamaño del conjunto de validación: (292, 6)


In [8]:
# Convertir el dataset a un dataframe de pandas y asignar los datos correspondientes a entrenamiento y validación
columnas = ["SalePrice", "OverallQual", "1stFlrSF", "TotRmsAbvGrd", "YearBuilt", "LotFrontage"]
df = pd.DataFrame(datos, columns=columnas)
df_train = pd.DataFrame(train_data, columns=columnas)
df_val = pd.DataFrame(val_data, columns=columnas)

# Mostrar primeras filas del conjunto de entrenamiento
print("Entrenamiento: \n", df_train.head())

# Mostrar primeras filas del conjunto de validación
print("Validación: \n", df_val.head())

Entrenamiento: 
    SalePrice  OverallQual  1stFlrSF  TotRmsAbvGrd  YearBuilt  LotFrontage
0   142000.0          5.0    1188.0           6.0     1959.0          NaN
1   146000.0          7.0     703.0           5.0     2007.0         34.0
2   256300.0          8.0    1600.0           7.0     2007.0        110.0
3   128000.0          6.0     958.0           5.0     1976.0          NaN
4    85000.0          5.0     536.0           4.0     1976.0         36.0
Validación: 
    SalePrice  OverallQual  1stFlrSF  TotRmsAbvGrd  YearBuilt  LotFrontage
0   147000.0          5.0     672.0           6.0     1920.0         59.0
1   266000.0          9.0    1684.0           6.0     2005.0         63.0
2   262280.0          7.0    1532.0          10.0     1990.0         86.0
3   221500.0          7.0    1614.0           7.0     2005.0         86.0
4   168000.0          4.0    1622.0           7.0     1961.0          NaN
